In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-08 03:21:44.787524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746688904.799532  430557 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746688904.803376  430557 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746688904.812626  430557 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746688904.812650  430557 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746688904.812651  430557 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [ ]:
### Specify Environment Configuration ###

# |Location to store simulation results
out_dir = "examples/CoDec_Research/results/simulation_results/Viz/"
simulation_results_files = [out_dir+fl_name for fl_name in listdir(out_dir)]

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal'

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
num_parallel_envs = 1
total_envs = 2
device = "cpu" # cpu just because we're in a notebook
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1      # Agents observed except self (used for vector sizes)
sample_size_traj = 1                             # Number of samples to calculate expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )


In [ ]:

### Generate Construed Agent Trajectories Through Simulator Sampling ###
traj_obs = None

2# |Check if saved data is available
for srFile in simulation_results_files:
    if "all_constr_obs_" in srFile:
        with open(srFile, 'rb') as opn_file:
            traj_obs = pickle.load(opn_file)
    else:
        continue

if traj_obs is None:
    default_values, traj_obs, ground_truth, veh_indx2ID = generate_all_construal_trajnval(
                                                                                            out_dir=out_dir,
                                                                                            sim_agent=sim_agent,
                                                                                            observed_agents_count=observed_agents_count,
                                                                                            construal_size=construal_size,
                                                                                            num_parallel_envs=num_parallel_envs,
                                                                                            max_agents=max_agents,
                                                                                            sample_size=sample_size_traj,
                                                                                            device=device,
                                                                                            train_loader=train_loader,
                                                                                            env=env,
                                                                                            env_multi_agent=env_multi_agent,
                                                                                            generate_animations=False,
                                                                                        )

Processing Waymo batches:   0%|          | 0/2 [00:00<?, ?it/s]

Indices of all moving vehicles (by scene):  [(0, 1, 5, 9, 10, 42, 43)]
	sample  1
		Step: 89
Processed mask(s):  ((0,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((1,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((5,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((9,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((10,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((42,),) , with value(s): [1.0]
	sample  1
		Step: 89
Processed mask(s):  ((43,),) , with value(s): [1.0]
	sample  1
		Step: 89

Processing Waymo batches:  50%|█████     | 1/2 [00:13<00:13, 13.39s/it]


Processed mask(s):  ((0, 1, 5, 9, 10, 42, 43),) , with value(s): [1.0]
Indices of all moving vehicles (by scene):  [(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17)]
	sample  1
		Step: 90
Processed mask(s):  ((0,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((1,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((2,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((3,),) , with value(s): [1.0]
	sample  1
		Step: 91
Processed mask(s):  ((4,),) , with value(s): [0.0]
	sample  1
		Step: 90
Processed mask(s):  ((5,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((7,),) , with value(s): [1.0]
	sample  1
		Step: 91
Processed mask(s):  ((8,),) , with value(s): [0.0]
	sample  1
		Step: 90
Processed mask(s):  ((9,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((10,),) , with value(s): [1.0]
	sample  1
		Step: 90
Processed mask(s):  ((11,),) , with value(s): [1.0]
	sample  1
		Step

Processing Waymo batches: 100%|██████████| 2/2 [00:39<00:00, 21.06s/it]


Processed mask(s):  ((0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17),) , with value(s): [1.0]


Processing Waymo batches: 100%|██████████| 2/2 [00:39<00:00, 19.91s/it]

Baseline data saved to:  examples/CoDec_Research/results/simulation_results/Viz/construal_vals_2025-05-08 03:22:26.257373.pickle
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/Viz/all_constr_obs_2025-05-08 03:22:26.257928.pickle
Baseline data saved to:  examples/CoDec_Research/results/simulation_results/Viz/ground_truth_2025-05-08 03:22:26.268233.pickle


In [4]:
veh_indx2ID

{'tfrecord-00101-of-01000_71': {0: 2892.0,
  1: 2459.0,
  5: 2464.0,
  9: 2468.0,
  10: 2469.0,
  42: 2525.0,
  43: 2526.0},
 'tfrecord-00117-of-01000_240': {0: 963.0,
  1: 597.0,
  2: 598.0,
  3: 600.0,
  4: 601.0,
  5: 604.0,
  7: 607.0,
  8: 609.0,
  9: 613.0,
  10: 615.0,
  11: 616.0,
  12: 617.0,
  13: 618.0,
  14: 619.0,
  15: 620.0,
  16: 623.0,
  17: 624.0}}

In [8]:
# |Extract plot data from observations
from gpudrive.datatypes.observation import PartnerObs


plot_data = {}
for scene, scene_obs in traj_obs.items():
    if scene == 'dict_structure':
        continue
    plot_data[scene] = {}
    for construal, construal_obs in scene_obs.items():
        plot_data[scene][construal] = {}
        for sample, sample_obs in construal_obs.items():
            plot_data[scene][construal][sample] = {}
            print(f"Scene: {scene}, Construal: {construal}, Sample: {sample}")
            print(sample_obs.shape)

Scene: tfrecord-00101-of-01000_71, Construal: (0,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (1,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (5,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (9,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (10,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (42,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (43,), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00101-of-01000_71, Construal: (0, 1, 5, 9, 10, 42, 43), Sample: 0
torch.Size([64, 89, 2])
Scene: tfrecord-00117-of-01000_240, Construal: (0,), Sample: 0
torch.Size([64, 90, 2])
Scene: tfrecord-00117-of-01000_240, Construal: (1,), Sample: 0
torch.Size([64, 90, 2])
Scene: tfrecord-00117-of-01000_240, Construal: (2,), Sample: 0
torch.Size([64, 90, 2])
Scene: tfrecord-00117-of-010

In [6]:
traj_obs.keys()

dict_keys(['dict_structure', 'tfrecord-00101-of-01000_71', 'tfrecord-00117-of-01000_240'])

# Server COnstrual Values

In [ ]:
srFile = "examples/CoDec_Research/results/simulation_results/construal_vals_server.pickle"
with open(srFile, 'rb') as opn_file:
    default_values = pickle.load(opn_file)